# LLM으로 유튜브 영상 TEXT 가져와서 번역 및 요약

* 라이브러리 : [youtube_transcript_api](https://pypi.org/project/youtube-transcript-api/)
* 참고 : [NLP와 LLM 실전 가이드(한빛미디어, 박조은)](https://github.com/corazzon)

In [3]:
# 필요한 주요 라이브러리 설치
!pip install -Uq langchain langchain_openai langchain_community youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


Imports:

In [4]:
import os
import textwrap
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

### 코드 설정

In [12]:
# 본인의 OpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

#### Youtube Video URL

In [6]:
from youtube_transcript_api import YouTubeTranscriptApi

ytt_api = YouTubeTranscriptApi()
video_url = "https://www.youtube.com/watch?v=H14bBuluwB8" # Grit: The Power of Passion and Perseverance | Angela Lee Duckworth | TED
video_id = video_url.split("v=")[-1]
fetched_transcript  = ytt_api.fetch(video_id,languages=['en'])

In [7]:
# is iterable
for snippet in fetched_transcript:
    print(snippet.text)

Transcriber: Joseph Geni
Reviewer: Morton Bast
When I was 27 years old,
I left a very demanding job
in management consulting
for a job that was even more
demanding: teaching.
I went to teach seventh graders math
in the New York City public schools.
And like any teacher,
I made quizzes and tests.
I gave out homework assignments.
When the work came back,
I calculated grades.
What struck me was that IQ
was not the only difference
between my best and my worst students.
Some of my strongest performers
did not have stratospheric IQ scores.
Some of my smartest kids
weren't doing so well.
And that got me thinking.
The kinds of things you need
to learn in seventh grade math,
sure, they're hard: ratios, decimals,
the area of a parallelogram.
But these concepts are not impossible,
and I was firmly convinced
that every one of my students
could learn the material
if they worked hard and long enough.
After several more years of teaching,
I came to the conclusion
that what we need in education
is a m

In [8]:
english_text = " ".join([snippet.text for snippet in fetched_transcript])
english_text

'Transcriber: Joseph Geni\nReviewer: Morton Bast When I was 27 years old, I left a very demanding job\nin management consulting for a job that was even more\ndemanding: teaching. I went to teach seventh graders math in the New York City public schools. And like any teacher,\nI made quizzes and tests. I gave out homework assignments. When the work came back,\nI calculated grades. What struck me was that IQ\nwas not the only difference between my best and my worst students. Some of my strongest performers\ndid not have stratospheric IQ scores. Some of my smartest kids\nweren\'t doing so well. And that got me thinking. The kinds of things you need\nto learn in seventh grade math, sure, they\'re hard: ratios, decimals,\nthe area of a parallelogram. But these concepts are not impossible, and I was firmly convinced\nthat every one of my students could learn the material if they worked hard and long enough. After several more years of teaching, I came to the conclusion\nthat what we need in e

## 검토, 요약, 다국어 번역

In [15]:
# 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini")

user_prompt_template = """
다음 지시를 순서대로 수행하세요.

1. 제공된 전체 텍스트를 주의 깊게 읽고, 핵심 내용을 5문장 길이의 영어 문장으로 요약합니다.
2. 요약문이 원문과 의미가 일치하는지 검토하고, 필요한 경우 정확성을 위해 수정합니다.
3. 완성된 영어 요약을 한국어, 중국어, 일본어 순으로 번역합니다.
4. 출력은 아래 구분 기호 형식을 정확히 지켜서 작성합니다.

[출력 형식]
영어 요약
----
한국어 번역
****
중국어 번역
====
일본어 번역

영어로 작성할 원문:
{content}
"""


# 프롬프트 정의
summary_prompt = PromptTemplate(template=user_prompt_template,
                                input_variables=["content"])

# 체인 구성
summary_chain = summary_prompt | llm

# 요약 및 번역 수행
prompt_input = summary_prompt.format(content=english_text)  # 프롬프트 텍스트로 변환
original_answer = summary_chain.invoke(prompt_input)       # 텍스트 입력
print(original_answer.content)

# 결과 출력
print("\n요약 및 번역 결과:\n")
print(textwrap.fill(
    str(original_answer.content), width=50, replace_whitespace=True))


영어 요약  
The speaker reflects on their transition from management consulting to teaching seventh graders in New York City, discovering that success in education is influenced by factors beyond IQ. This realization led them to pursue a career in psychology, focusing on understanding what contributes to success in various challenging environments. Their research highlighted that "grit," defined as passion and perseverance for long-term goals, emerged as a significant predictor of success. Notably, students with higher grit levels were more likely to graduate, regardless of other factors. The speaker emphasizes the importance of promoting grit and acknowledges the need for further exploration on how to effectively cultivate it in children.

----
한국어 번역  
강연자는 경영 컨설팅에서 뉴욕시의 7학년 학생들을 가르치는 일로 전환한 경험을 반영하며, 교육에서 성공은 IQ 이상의 여러 요인에 영향을 받는다고 발견했습니다. 이 깨달음은 그들을 심리학 분야로 나아가게 하였고, 다양한 도전적인 환경에서 성공의 요인이 무엇인지 이해하는 데 집중하게 했습니다. 그들의 연구 결과 "그릿" 즉, 장기적인 목표를 위한 열정과 인내가 성공의 중요한 예측 요소로 떠올랐습니다. 특히, 그릿이 높은 학생들

In [14]:
# 주요 요점 추출 프롬프트 템플릿 정의
system_prompt_template = """
당신은 간결하고 명확한 요점을 제공하는 전문가입니다.
이전 질문에 대한 답변은 다음과 같습니다: <{original_answer}>
이 한국어 답변을 3-5개의 주요 요점으로 정리해 주세요.
각 요점은 글머리 기호로 시작하는 한국어로만 작성해 주세요.
"""
system_prompt_template = PromptTemplate(template=system_prompt_template,
                                   input_variables=["original_answer"])

# 체인 구성 (RunnableSequence 사용)
summary_chain = system_prompt_template | llm

# 요점 추출 수행
summary = summary_chain.invoke({"original_answer": original_answer})  # .invoke 사용

# 결과 출력
print("\n한국어 주요 요점:\n")
print(summary.content)


한국어 주요 요점:

- 저자는 27세에 경영 컨설팅에서 뉴욕 7학년 수학 교육으로 전환하며 IQ만으로 학업 성취를 설명할 수 없음을 깨달았다.
- 이 깨달음은 동기와 심리학 이해에 집중하게 했고, 이후 심리학자로서 성공의 예측 요소를 연구하였다.
- 연구 결과, 장기 목표에 대한 열정과 인내인 '그릿'이 성공의 핵심 요소임을 발견했으며, 높은 그릿 수준을 가진 학생들이 졸업할 가능성이 더 높았다.
- 그릿의 중요성 인식에도 불구하고, 저자는 이를 배양하는 방법에 대한 과학적 이해가 더 필요하다고 강조했다.
- 저자는 성장 마인드셋이 그릿을 발전시키는 데 도움이 될 수 있다고 제안했다.
